# ZBIW Data Librarian Modul 2 - Präsenztag

## Umlaut-Problematik

Allgemeine Infos: https://docs.python.org/3/howto/unicode.html

### Urllib

Urllib kann nicht mit Umlauten oder anderen speziellen Sonderzeichen in der URL umgehen, daher muss bei Vorhandensein die URL vorher umgewandelt werden. Das geht beispielsweise mit der Funktion `urllib.parse.quote`.

Link zur Doku: https://docs.python.org/3/library/urllib.parse.html#url-quoting

Wenn das Ergebnis vom Request empfangen wird, wird der `decode` Funktion explizit das Encoding als Parameter mitgegeben, um korrekt encodete Daten zu erhalten.

Die `decode` Funktion ist Teil der Standard-Library: https://docs.python.org/3/library/stdtypes.html#bytes.decode

In [ ]:
import json
import urllib.request
import urllib.parse

# Umlaute in der URL:
url = "https://www.bibsonomy.org/json/search/" + parse.quote("Lösungen") + "?items=1000&duplicates=merged"

f = urllib.request.urlopen(url)
print(type(f))

In [ ]:
read = f.read() # read() ist Methode des HTTPResponse-Objektes
print(type(read)) # Rückgabe-Typ = bytes

In [ ]:
result = read.decode('utf-8') # Decoden der Bytes in einen str

# Ergebnis ist str, muss erst noch als JSON verarbeitet werden (Ergebnis = dict)
data = json.loads(result) # json.loads() lädt JSON Daten aus einem String
data

### Requests

https://2.python-requests.org/en/master/user/quickstart/#response-content

"When you make a request, Requests makes educated guesses about the encoding of the response based on the HTTP headers. The text encoding guessed by Requests is used when you access `r.text`. You can find out what encoding Requests is using, and change it, using the `r.encoding` property:"

In [ ]:
# requests hat kein Problem mit Umlauten in URLs
url = "https://www.bibsonomy.org/json/search/Lösungen?items=1000&duplicates=merged"

In [ ]:
import requests

result = requests.get(url) # result ist ein requests.models.Response Objekt
result.encoding # Das Encoding der Response lässt sich abfragen

In [ ]:
# Encoding lässt sich auch ändern:
result.encoding = 'ISO-8859-1'

In [ ]:
data = result.json()
print(data)

---

## Crossref-Fragen

### Facets

Facets sind dazu da, Ergebnislisten nach Facetten aufzuteilen.

Im konkreten Beispiel der Facette "published" besteht die Semantik darin, dass man die maximale Anzahl der zurückzuliefernden Facetten angibt. Die Angabe `facet=published:2015` bedeutet also nicht, dass man alle Publikationen aus 2015 erhält, sondern dass im Ergebnis maximal 2015 Facetten aufgelistet werden. Der Maximalwert ist aber nur 51.

Die Ergebnismenge wird durch die Facettenangabe nicht beeinflusst.

Siehe Doku: https://github.com/CrossRef/rest-api-doc#facet-counts

In [ ]:
url = "https://api.crossref.org/works?query.author=herpers&rows=1000&facet=published:2015"

In [ ]:
result = requests.get(url)
data = result.json()
data['message']['total-results']

In [ ]:
url = "https://api.crossref.org/works?query.author=herpers&rows=1000"
result = requests.get(url)
data = result.json()
data['message']['total-results']

Filter demgegenüber werden verwendet, um die Ergebnisse nach bestimmten Kriterien zu filtern. Es gibt eine ganze Menge verfügbarer Filter, siehe Doku: https://github.com/CrossRef/rest-api-doc#filter-names

Hier ein Beispiel, mit dem die Ergebnisliste korrekt nach Jahreszahl des Publikationsjahres gefiltert wird:

In [ ]:
url = "https://api.crossref.org/works?query.author=herpers&rows=1000&filter=from-pub-date:2015,until-pub-date:2015"
result = requests.get(url)
data = result.json()
data['message']['total-results']

### Etiquette

Generell ist davon abzuraten, einen fremden Webserver mit allzu häufigen Anfragen zu bombardieren. Vermeidungsstrategien sind hierbei, wie auch in der Crossref-Etiquette angegeben, gezielte Pausen zwischen Abfragen, sowie Caching. Caching ist insbesondere bei wiederholt gleichen Anfragen anzuraten. Bei verschiedenen Anfragen sollte zwischen jeder Anfrage eine kleine Pause eingehalten werden. Hierfür gibt es keine festen Vorschriften, eine Minute sollte ausreichend sein.

---

## OAI-PMH APIs mit Python abfragen

In [ ]:
#import sys
#!conda install --yes --prefix {sys.prefix} -c auto pyoai
#!{sys.executable} -m pip install pyoai

In [ ]:
OAI_PHM_URL = 'https://www.ssoar.info/OAIHandler/request'

In [ ]:
from oaipmh.client import Client
from oaipmh.metadata import MetadataRegistry, oai_dc_reader

registry = MetadataRegistry()
registry.registerReader('oai_dc', oai_dc_reader)
client = Client(OAI_PHM_URL, registry)

In [ ]:
# Get single record
record = client.getRecord(metadataPrefix='oai_dc', identifier='oai:gesis.izsoz.de:document/679')
header = record[0]
print('id: {}'.format(header.identifier()))
print('element: {}'.format(header.element()))
print('datestamp: {}'.format(header.datestamp()))

In [ ]:
metadata = record[1].getMap()
if 'creator' in metadata:
    print('creator: {}'.format(metadata['creator']))

In [ ]:
# List records
for record in client.listRecords(metadataPrefix='oai_dc'):
    header = record[0]
    print('id: {}'.format(header.identifier()))
    metadata = record[1].getMap()
    if 'creator' in metadata:
        print('creator: {}'.format(metadata['creator']))
    if 'title' in metadata:
        print('title: {}'.format(metadata['title']))
    print()

---

## XML Verarbeitung

Es gibt viele Libraries, mit denen man in Python XML verarbeiten kann. Eine Liste:

* xml.dom ([Standard](https://docs.python.org/3.5/library/xml.dom.html#)) 
* xml.etree ([Standard](https://docs.python.org/3.8/library/xml.etree.elementtree.html#))              
* libxml2 (C extension)
* BeautifulSoup ([Extern](https://www.crummy.com/software/BeautifulSoup/bs4/doc/))

Im Folgenden ein kleines Beispiel mit den Möglichkeiten der Standard-Library.

Doku: https://docs.python.org/3.8/library/xml.etree.elementtree.html#

In [ ]:
url = "https://www.bibsonomy.org/layout/dblp/search/Bibliothek?items=100"

import requests
result = requests.get(url)

type(result)

In [ ]:
result.text

In [ ]:
from xml.etree import ElementTree as ET
from xml.etree.ElementTree import fromstring

tree = ET.ElementTree(fromstring(result.text))
root = tree.getroot()
root

In [ ]:
for book in root.findall('book'):
    key = book.get('key')
    print(key)
    title = book.find('title').text
    print(title)
    print()